In [ ]:
%uv sync

In [ ]:
from dotenv import load_dotenv
from azure.search.documents.indexes.aio import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex,        
    SearchFieldDataType
)
import os
import requests

In [5]:
load_dotenv(override=True)

search_endpoint = os.getenv('SEARCH_ENDPOINT')
search_api_key = os.getenv('SEARCH_API_KEY')

In [2]:
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="vector-profile-lore",  
            algorithm_configuration_name="myHnsw"
        )
    ]
)

In [ ]:
index = {
        'name': 'lore',
        'fields': [
                SearchField(name="id", type=SearchFieldDataType.String,key=True),                   
                SearchField(name="type", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=True),                      
                SearchField(name="title", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=False, filterable=False),                  
                SearchField(name="content", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=False, filterable=False),                
                SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1024, vector_search_profile_name="vector-profile-lore",searchable=True,sortable=False, facetable=False, filterable=False),
                SearchField(name="category", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=False),
                SearchField(name="level_requierement", type=SearchFieldDataType.Int32, searchable=True,sortable=True, facetable=False, filterable=False), 
                SearchField(name="prerequisites", type=SearchFieldDataType.Int32, searchable=True,sortable=True, facetable=False, filterable=False),     
        ]    
} 


In [6]:
index_client = SearchIndexClient(endpoint=search_endpoint,credential=AzureKeyCredential(search_api_key))

In [ ]:
index = SearchIndex(name=index['name'], fields=index['fields'], vector_search=vector_search)

result = await index_client.create_or_update_index(index=index)

In [ ]:
print(result)

In [18]:
from pydantic import BaseModel

class VectorSetting(BaseModel):
    algorithm_name: str
    vector_profile_name: str
    vector_algorithm_name: str

In [16]:
import json

with open('vector.json', 'r') as file:
    data = json.load(file)

vector_settings = [VectorSetting(**item) for item in data]    

In [19]:
def find_vector_profile(profile_name) -> VectorSetting | None:
   return next((s for s in vector_settings if s.vector_profile_name == profile_name))


In [ ]:
result = find_vector_profile("vector-profile-lore")

print(result)

In [21]:
with open('index.json', 'r') as file:
    indexes = json.load(file)

In [ ]:
headers = {
    'Content-Type': 'application/json',
    'api-key': search_api_key
}

url = f'{search_endpoint}/indexes?api-version=2025-09-01'

for idx in indexes:
    response = requests.post(url=url,headers=headers,data=json.dumps(idx))

    if response.status_code == 200 or response.status_code == 201:
        result = response.json()
        print(result)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)